# EMERFIN DASHBOARD
Python 2.7 script to generate charts and latex code

In [1]:
# Import preliminaries
import requests
import pandas as pd
import time
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import fnmatch
import numpy as np

In [8]:
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
series_url = '{}CodeList/CL_INDICATOR_IFS'.format(url)
area_url = '{}CodeList/CL_AREA_IFS'.format(url)
data_url= '{}CompactData/IFS/'.format(url)
ds_url = '{}DataStructure/IFS'.format(url)
SearchTerms = ['ic Product, Real','mploym','Industrial Pro','ices, Net','Interest Rate','Price In']
CountryList = ['BR','RU','IN','CN','ZA','VN','MX','ID','NG','TR','RO','AR','CO','EG','TH','PL',
               'PH','BG','FR','DE','US','JP','SN','MU','KE','UA','IT','GR','MY','CL','JM','MA',
               'PE','AU','AT','CA','CR','HU','DZ','IE','PT','ES','NL','KR','UY','BH','SA','PY',
               'HN','MN','OM','QA','SV','SK','LK','FJ','RS','NO','DK','LT','MZ','NZ','KW','KH',
               'GH','EC','PA','NP','BO','CH','PK','SG','FI','SI','SE','LV','BD','GB'
              ]# ['BH','OM']#
dfcols = ['@FREQ','@INDICATOR','@REF_AREA','@UNIT_MULT', 'Obs','@TIME_FORMAT']
name_vars = {'Short':2,'Full':3,'Concept':5,'Unit_Name':7,'Topic':8}

In [9]:
# Request r_list of IFS series that meet SearchTerms
s_list = requests.get(series_url).json()['Structure']['CodeLists']['CodeList']['Code']
s_dict = {s_list[i]['@value']: s_list[i]['Description']['#text'] for i in range(0,len(s_list))}
r_list = [k for k, v in s_dict.items() if any([x in v for x in SearchTerms])]

In [4]:
# Final results must have data available for r_list and CountryList 
ResultsList = []
for r in r_list:
    url_r = '{}.{}.{}.?startPeriod=2013'.format(data_url, '+'.join(CountryList), r)
#    print url_r    
    data = requests.get(url_r).json()['CompactData']['DataSet']
    if 'Series' in data.keys():
        for n in range(0,len(data['Series'])):
            if 'Obs' in data['Series'][n].keys(): 
                ResultsList.append(data['Series'][n])
    time.sleep(0.5)   ## Delay requests slightly so that IMF does not block them

In [5]:
# Generate .csv of best series
df = pd.DataFrame(ResultsList, columns=(dfcols)).iloc[:,0:4]
df['Freq'] = df['@FREQ'].replace({'M':2, 'Q':1, 'A':0})  ## Monthly data pref
df['Combined'] = df['@REF_AREA'].str.cat(df['@INDICATOR'], sep='.')
df = df.sort_values(['Combined', 'Freq']).drop_duplicates('Combined', keep='last')
df['Title'] = df['@INDICATOR'].map(s_dict)
df['Source'] = 'IFS'
## Country Full Names
s_area = requests.get(area_url).json()['Structure']['CodeLists']['CodeList']['Code']
d_area = {s_area[i]['@value']: s_area[i]['Description']['#text'] for i in range(0,len(s_area))}
df['Country'] = df['@REF_AREA'].map(d_area)
## Unit Multiplier Text
ds = requests.get(ds_url).json()['Structure']['CodeLists']['CodeList'][0]['Code']
d_unit = {ds[i]['@value']: ds[i]['Description']['#text'] for i in range(0,len(ds))}
df['Units'] = df[df['@UNIT_MULT']!='0']['@UNIT_MULT'].map(d_unit)
## Additional Text
df2 = pd.DataFrame(requests.get(ds_url).json()['Structure']['CodeLists']['CodeList'][3]['Code'])
df2 = df2[df2['@value'].isin(pd.unique(df['@INDICATOR']))]
for k, v in name_vars.items():
    d = {}
    for i in range(0,len(df2)):
        d[df2.iloc[i]['@value']] = df2.iloc[i]['Annotations']['Annotation'][v]['AnnotationText']['#text']
    df[k] = df['@INDICATOR'].map(d)   
df['Y_Label'] = df['Unit_Name'].str.cat(df['Units'], sep=', ').fillna(df['Unit_Name'])
df['Combined'] = df['@FREQ'].str.cat(df['Combined'], sep='.')
df = df.set_index('Combined')
df['Category'] = df['Full'].str.split(',', expand=True, n=1)[0]
df['Chart_Title'] = df['Title'].str.rsplit(',', expand=True, n=1)[0]
f1 = 'Percentage change, corresponding period previous year'
f2 = 'Percent Change, Corresponding Period Previous Year'
r1 = 'Annual percentage change'
df['Chart_Title'] = df['Chart_Title'].str.replace(f1, r1)
df['Chart_Title'] = df['Chart_Title'].str.replace(f2, r1)
df = df.sort_values(['Country', 'Category', 'Chart_Title'])

In [6]:
# Categorize each chart into one of 8 groups
Group_dict = {
    'National Accounts':'1-National Accounts',
    'Labor Markets':'2-Labor Markets',
    'Social Indicators':'2-Labor Markets',
    'Economic Activity':'3-Economic Activity',
    'Producer Price Index':'4-Prices',
    'Consumer Prices':'4-Prices',
    'Prices':'4-Prices',
    'Balance of Payments':'5-Balance of Payments',
    'Exchange Rates':'6-Exchange Rates',
    #:'7-Financial Policy Rate',  # Find from Chart_Title later
    'Financial':'8-Financial Market Rate'
}
Policy_kw = ['Policy', 'Lending', 'Discount']
df['Group'] = df['Category'].map(Group_dict)
df.loc[df['Chart_Title'].str.contains('Pol|Disc').tolist(), 'Group'] = '7-Financial Policy Rate'

In [7]:
# Source
df['Source'] = 'IFS'
# Regions Mapping from CSV file
Regions = pd.read_csv('regions.csv', index_col='country').to_dict()['region']
df['Region'] = df['Country'].map(Regions)
# Chart Type
df['Type'] = 'plot'
df.loc[df['Group'].str.contains('1|5').tolist(), 'Type'] = 'bar'

In [8]:
# Main or Alt
df['Main_Alt'] = 'Alternate'
df.loc[df['Title'].str.contains('Real, P|eal Eff|Cons.*cent|ment Rate, P|ices, Net|Indus.*erc').tolist(), 'Main_Alt'] = 'Main'
for c in pd.unique(df['Country'].values):
    if len(df[df['Country']==c][(df['Group'].str.contains('3').tolist()) & (df['Main_Alt'] == 'Main')]) == 0:
        df.loc[(df['Country']==c) & (df['@INDICATOR'].str.contains('AIP_IX').tolist()), 'Main_Alt'] = 'Main'
        if len(df[df['Country']==c][(df['Group'].str.contains('3').tolist()) & (df['Main_Alt'] == 'Main')]) == 0:
            df.loc[(df['Country']==c) & (df['@INDICATOR'].str.contains('AIP_SA_IX').tolist()), 'Main_Alt'] = 'Main'
            if len(df[df['Country']==c][(df['Group'].str.contains('3').tolist()) & (df['Main_Alt'] == 'Main')]) == 0:
                df.loc[(df['Country']==c) & (df['@INDICATOR'].str.contains('AIPMA_IX').tolist()), 'Main_Alt'] = 'Main' 
                if len(df[df['Country']==c][(df['Group'].str.contains('3').tolist()) & (df['Main_Alt'] == 'Main')]) == 0:
                    df.loc[(df['Country']==c) & (df['@INDICATOR'].str.contains('AIPMA_SA_IX').tolist()), 'Main_Alt'] = 'Main'
    if len(df[df['Country']==c][(df['Group'].str.contains('7').tolist()) & (df['Main_Alt'] == 'Main')]) == 0:
        df.loc[(df['Country']==c) & (df['Title'].str.contains('Policy').tolist()), 'Main_Alt'] = 'Main'
        if len(df[df['Country']==c][(df['Group'].str.contains('7').tolist()) & (df['Main_Alt'] == 'Main')]) == 0:
            df.loc[(df['Country']==c) & (df['Title'].str.contains('Discount').tolist()), 'Main_Alt'] = 'Main'
    if len(df[df['Country']==c][(df['Group'].str.contains('8').tolist()) & (df['Main_Alt'] == 'Main')]) == 0:
        df.loc[(df['Country']==c) & (df['Title'].str.contains('ment Bonds, Per').tolist()), 'Main_Alt'] = 'Main'
        if len(df[df['Country']==c][(df['Group'].str.contains('8').tolist()) & (df['Main_Alt'] == 'Main')]) == 0:
            df.loc[(df['Country']==c) & (df['Title'].str.contains('Treas').tolist()), 'Main_Alt'] = 'Main'
            if len(df[df['Country']==c][(df['Group'].str.contains('8').tolist()) & (df['Main_Alt'] == 'Main')]) == 0:
                df.loc[(df['Country']==c) & (df['Title'].str.contains('Lending Rate, P').tolist()), 'Main_Alt'] = 'Main' 

C:\Users\BDew\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\BDew\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\BDew\Anaconda2\lib\site-packages\ipykernel\__main__.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\BDew\Anaconda2\lib\site-packages\ipykernel\__main__.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\BDew\Anaconda2\lib\site-packages\ipykernel\__main__.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\BDew\Anaconda2\lib\site-packages\ipykernel\__main__.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\BDew\Anaconda2\lib\site-packages\ipykernel\__main__.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame i

In [9]:
df.to_csv('IFS_Series_List.csv')

In [10]:
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/IFS/'
series_df = pd.read_csv('IFS_Series_List.csv', index_col='Combined') #df.sort_values(['Country', 'Group']) #
s_list = series_df.index[series_df['Main_Alt'] == 'Main']
color_list = {
    1: u'red',
    2: u'blue', 
    3: u'#2ecc71', 
    4: u'#e67e22', 
    5: u'#5dade2',
    6: u'#a569bd',
    7: u'#1c7c45',
    8: u'#ffc300',
}
yearsFmt = mdates.DateFormatter('%Y')

In [11]:
for v in s_list: #[s_list[0]]
    data = requests.get('{}{}.?startPeriod=2006'.format(url, v)).json()['CompactData']['DataSet']['Series']['Obs']
    time.sleep(0.5)   ## Delay requests slightly so that IMF does not block them
    # Load data into a pandas dataframe and set index
    df = pd.DataFrame(data)
    df = df.set_index(pd.to_datetime(df['@TIME_PERIOD']))
    plttype = series_df.loc[v]['Type']
    # Basic plot 
    ax = plt.gca()
    ax.yaxis.grid(True)
    ax.set_axisbelow(True)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.yaxis.set_ticks_position('left') 
    ax.xaxis.set_ticks_position('bottom')
    ax.xaxis.set_major_formatter(yearsFmt)
    ax.xaxis.set_label_coords(0.35, -0.1)
    ax.margins(0.05)
    color_id = color_list[int(series_df.loc[v]['Group'][0])]
    if 'plot' in series_df.loc[v]['Type']:
        plt.plot(df['@OBS_VALUE'].astype(float), linewidth=2, color=color_id)
    else:
        if v in fnmatch.filter(s_list, 'A*GDP*'):
            plt.bar(df.index, df['@OBS_VALUE'].astype(float), width=180, color=color_id, linewidth=0)
        if v in fnmatch.filter(s_list, 'A*BGS*'):
            plt.bar(df.index, df['@OBS_VALUE'].astype(float), width=180, color=color_id, linewidth=0)
        else: 
            plt.bar(df.index, df['@OBS_VALUE'].astype(float), width=60, color=color_id, linewidth=0)
    plt.tick_params(axis='both', which='major', labelsize=7)
    plt.tick_params(axis='both', which='minor', labelsize=7)
    plt.title(series_df.loc[v]['Chart_Title'], fontsize=10)
    if len(series_df.loc[v]['Chart_Title']) <= 50:
        plt.title(series_df.loc[v]['Chart_Title'], fontsize=12)
    plt.rcParams['figure.figsize'] = (5, 3)
    plt.ylabel(series_df.loc[v]['Y_Label'], fontsize=8)
    if 'Index' in series_df.loc[v]['Y_Label']:
        if '@BASE_YEAR' in data[0].keys():
#            print v
            plt.ylabel('{}, ({}=100)'.format(series_df.loc[v]['Y_Label'], data[0]['@BASE_YEAR']), fontsize=8)
    lvv = np.round(float(df['@OBS_VALUE'][-1]), decimals=2)
    lvd = df.index[-1].strftime('%Y-%m-%d')    
    plt.xlabel('Source: IFS      Latest Value: {}: {}'.format(lvd, lvv), fontsize=9)
    plt.savefig('plots/{}_{}_v1.pdf'.format(v[2:4],v[5:]), bbox_inches='tight')
    plt.close()

In [12]:
series_df = pd.read_csv('IFS_Series_List.csv', index_col='Combined').sort_values(['Region','Country', 'Group'])
s_list = series_df.index[series_df['Main_Alt'] == 'Main']
ltx1 = '\includegraphics[max width=0.5\\textwidth]{'
ltx2 ='\\thispagestyle{main} \\noindent '
for r in pd.unique(series_df['Region']):
    print '\section*{{{}}}'.format(r)
    print '\markboth{{{}}}{{}}'.format(r)
    for c in pd.unique(series_df.loc[series_df['Region']==r, 'Country']):
        print '\subsection*{{{}}}{}'.format(c, ltx2)
        idx = series_df[(series_df['Main_Alt'] == 'Main') & (series_df['Country'] == str(c))].index
        for v in range(0,len(idx)):
            print '{}{}_{}_v1.pdf}}'.format(ltx1, idx[v][2:4],idx[v][5:])
        print '\\newpage'

\section*{Africa}
\markboth{Africa}{}
\subsection*{Ghana}\thispagestyle{main} \noindent 
\includegraphics[max width=0.5\textwidth]{GH_PCPI_PC_CP_A_PT_v1.pdf}
\includegraphics[max width=0.5\textwidth]{GH_BGS_BP6_USD_v1.pdf}
\includegraphics[max width=0.5\textwidth]{GH_EREER_IX_v1.pdf}
\includegraphics[max width=0.5\textwidth]{GH_FPOLM_PA_v1.pdf}
\includegraphics[max width=0.5\textwidth]{GH_FIGB_PA_v1.pdf}
\newpage
\subsection*{Kenya}\thispagestyle{main} \noindent 
\includegraphics[max width=0.5\textwidth]{KE_NGDP_R_PT_v1.pdf}
\includegraphics[max width=0.5\textwidth]{KE_PCPI_PC_CP_A_PT_v1.pdf}
\includegraphics[max width=0.5\textwidth]{KE_BGS_BP6_USD_v1.pdf}
\includegraphics[max width=0.5\textwidth]{KE_FPOLM_PA_v1.pdf}
\includegraphics[max width=0.5\textwidth]{KE_FITB_PA_v1.pdf}
\newpage
\subsection*{Mauritius}\thispagestyle{main} \noindent 
\includegraphics[max width=0.5\textwidth]{MU_NGDP_R_PT_v1.pdf}
\includegraphics[max width=0.5\textwidth]{MU_LUR_PT_v1.pdf}
\includegraphics[max widt